### **INTRODUÇÃO**

Na atividade a seguir, o objetivo foi de implementar três novas funções de ativação diferentes da sigmoide aneteriormente utilizada — *ReLU (Rectified Linear Unit)*, *ELU (Exponential Linear Unit)* e *Swish*, nesse caso — na estrutura de rede neural construída em Python puro durante a disciplina. Cada função foi incorporada manualmente a rede de modo a substituir a função sigmoidal padrão e observar seu impacto no aprendizado da rede por meio do treino em épocas.

----

### **TEORIA DAS FUNÇÕES**

#### ***RELU (Rectified Linear Unit)***

A função de ativação ReLU (ou Rectified Linear Unit) é relativamente simples no seu âmbito matemático, dado que não envolve nenhum operação complexa durante todo o processo na rede. 

Em resumo, a rede atua zerando os valores de entrada quando negativos e os retorna quando positivos. A função pode ser vista abaixo:


$$
\text{ReLU}(x) =
\begin{cases}
x, & \text{se } x > 0 \\
0, & \text{se } x \leq 0
\end{cases}
$$

No caso da ReLU, há um problema que pode ser encontrado decorrente dessa relação com os valores negativo: a presença de "neurônios mortos". Esses neurônios são decorrentes de uma possível entrada de somente valores negativos, de modo que a derivada desses neurônios sempre vai ser igual a 0, inativando-os.  

#### ***ELU (Exponential Linear Unit)***

A função de ativação ELU (ou Exponential Linear Unit) segue um princípio parecido se comparada com a ReLU, seguindo inclusive o exato mesmo comportamento para valores positivos. Entretanto, para valores negativos, a função segue um decaimento exponencial, o que possibilita impedir o problema dos neurônios mortos encontrados na função ReLU. A função de ativação pode ser dada pela relação abaixo:

$$
\text{ELU}(x) =
\begin{cases}
x, & \text{se } x > 0 \\
\alpha(e^x - 1), & \text{se } x \leq 0
\end{cases}
$$

#### ***Swish***

No caso da função Swish, são permitidos os valores negativos e não existe uma diferença exata na relação sobre os valores em relação a 0, de forma que não há condições específicas para os valores se eles são positivos ou negativos. 

A função busca combinar os próprios valores recebidos com a sua sigmoide, com o valor de ativação sendo assim suavemente ajustado. Possui suas certas semelhanças com a função ReLU, mas por permitir os valores negativos, atua de uma forma contínua. A relação pode ser dada pela fórmula matemática abaixo:

$$
\text{Swish}(x) = x \cdot \text{sig}(x) = x \cdot \frac{1}{1 + e^{-x}}
$$

---

### **CÓDIGO**

Importamos primeiramente as bibliotecas que serão utilizadas:

In [1]:
import math
import random

#### ***CLASSES:***

Criamos as classes necessárias (Valor, Neurônio, Camada e MLP) para realizar uma rede neural MLP em Python Puro (já vistas em aula) com a adição dos métodos associados às novas funções de ativação que serão utilizadas durante o algoritmo:

In [2]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado
    
    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __rsub__(self, outro_valor):
        """Realiza a operação: outro_valor - self"""
        return Valor(outro_valor) - self
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    def sig(self):
        """Realiza a operação: exp(self) / (exp(self) + 1)"""
        return self.exp() / (self.exp() + 1)
    
    ### TRÊS NOVAS FUNÇÕES DE ATIVAÇÃO (ReLU, ELU e Swish)

    def relu(self):
        """Realiza a operação: ReLU(self)."""
        if self.data > 0:
            data = self.data
        else:
            data = 0
        progenitor = (self,)
        operador_mae = "ReLU"
        resultado = Valor(data, progenitor, operador_mae)

        def propagar_relu():
            if self.data > 0:
                self.grad += resultado.grad 
            else:
                pass
 
        resultado.propagar = propagar_relu
        return resultado

    def elu(self, alpha=1.0):
        """Realiza a operação: ELU(self, alpha)."""
        if self.data > 0:
            data = self.data
        else:
            data = alpha * (math.exp(self.data) - 1)
        progenitor = (self,)
        operador_mae = "ELU"
        resultado = Valor(data, progenitor, operador_mae)

        def propagar_elu():
            if self.data > 0:
                derivada = 1
            else:
                derivada = alpha * math.exp(self.data)
            self.grad += resultado.grad * derivada

        resultado.propagar = propagar_elu
        return resultado

    def swish(self):
        """Realiza a operação: Swish(self)."""
        sig = self.sig()
        resultado = self * sig

        def propagar_swish():
            self.grad += resultado.grad * (sig.data + self.data * sig.data * (1 - sig.data))

        resultado.propagar = propagar_swish
        return resultado
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

In [3]:
class Neuronio:
    def __init__(self, num_dados_entrada, fn_ativacao):
        self.vies = Valor(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))
        self.fn_ativacao = fn_ativacao
            
    def __call__(self, x):
        
        assert len(x) == len(self.pesos)
        
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  
        
        # alteramos para que o dado de saída seja associado à função escolhida no momento e não mais só à sigmóide
        
        ###dado_de_saida = soma.sig()
        if self.fn_ativacao == "sig":
            dado_de_saida = soma.sig()
        elif self.fn_ativacao == "relu":
            dado_de_saida = soma.relu()
        elif self.fn_ativacao == "elu":
            dado_de_saida = soma.elu()
        elif self.fn_ativacao == "swish":
            dado_de_saida = soma.swish()
                
        return dado_de_saida       
    
    def parametros(self):
        return self.pesos + [self.vies]

In [4]:
class Camada:
    def __init__(self, num_neuronios, num_dados_entrada, fn_ativacao):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada, fn_ativacao)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

In [5]:
class MLP:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada, fn_ativacao):
        
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i], fn_ativacao)
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x):
        for camada in self.camadas:
            x = camada(x)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params

#### ***TESTES:***

E assim, com os valores de x e y, podemos fazer os treinamento com os valores de previsão para cada uma das funções de ativação utilizadas baseado nas épocas de treinamento

In [6]:
x = [
    [1.2, -0.5, 2.0],
    [-1.0, 1.5, -0.8],
    [0.8, -0.3, 1.2],
    [1.5, 0.3, -1.0],
    [2.1, -1.3, 0.7]
]

y_true = [0.8, 0.1, 0.5, 0.2, 1.1]

NUM_DADOS_DE_ENTRADA = 3  
NUM_DADOS_DE_SAIDA = 1    
CAMADAS_OCULTAS = [3, 2]  

arquitetura_da_rede = CAMADAS_OCULTAS + [NUM_DADOS_DE_SAIDA]

##### TESTE COM RELU:

In [7]:
random.seed(8)
minha_mlp_relu = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede, fn_ativacao="relu")

In [8]:
NUM_EPOCAS = 200
TAXA_DE_APRENDIZADO = 0.2

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in x:
        previsao = minha_mlp_relu(exemplo)
        y_pred.append(previsao)

    # loss
    erros = []
    for yt, yp in zip(y_true, y_pred):
        residuo = yp - yt
        erro_quadratico = residuo ** 2
        erros.append(erro_quadratico)        
    loss = (sum(erros))/len(erros)

    # zero grad
    for p in minha_mlp_relu.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in minha_mlp_relu.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

    # mostra resultado (opcional)
    print(epoca, loss.data)


0 0.4300000000000001
1 0.4300000000000001
2 0.4300000000000001
3 0.4300000000000001
4 0.4300000000000001
5 0.4300000000000001
6 0.4300000000000001
7 0.4300000000000001
8 0.4300000000000001
9 0.4300000000000001
10 0.4300000000000001
11 0.4300000000000001
12 0.4300000000000001
13 0.4300000000000001
14 0.4300000000000001
15 0.4300000000000001
16 0.4300000000000001
17 0.4300000000000001
18 0.4300000000000001
19 0.4300000000000001
20 0.4300000000000001
21 0.4300000000000001
22 0.4300000000000001
23 0.4300000000000001
24 0.4300000000000001
25 0.4300000000000001
26 0.4300000000000001
27 0.4300000000000001
28 0.4300000000000001
29 0.4300000000000001
30 0.4300000000000001
31 0.4300000000000001
32 0.4300000000000001
33 0.4300000000000001
34 0.4300000000000001
35 0.4300000000000001
36 0.4300000000000001
37 0.4300000000000001
38 0.4300000000000001
39 0.4300000000000001
40 0.4300000000000001
41 0.4300000000000001
42 0.4300000000000001
43 0.4300000000000001
44 0.4300000000000001
45 0.430000000000000

##### TESTE COM ELU:

In [9]:
random.seed(8)
minha_mlp_elu = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede, fn_ativacao="elu")

In [10]:
NUM_EPOCAS = 200
TAXA_DE_APRENDIZADO = 0.5

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in x:
        previsao = minha_mlp_elu(exemplo)
        y_pred.append(previsao)

    # loss
    erros = []
    for yt, yp in zip(y_true, y_pred):
        residuo = yp - yt
        erro_quadratico = residuo ** 2
        erros.append(erro_quadratico)        
    loss = (sum(erros))/len(erros)

    # zero grad
    for p in minha_mlp_elu.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in minha_mlp_elu.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

    # mostra resultado (opcional)
    print(epoca, loss.data)

0 1.2331156539577761
1 0.12322279004712496
2 0.06867608106389582
3 0.04339159023082039
4 0.032683344340618176
5 0.02531253028229287
6 0.019460005871904565
7 0.01734967332768972
8 0.019114868507831936
9 0.027844699475921688
10 0.05047882188668168
11 0.09189608709583783
12 0.13194935124839863
13 0.166375727302536
14 0.11780457746728226
15 0.10598079099575855
16 0.06883665208274006
17 0.06324671845178527
18 0.05090183538259163
19 0.051012630676509546
20 0.04624020733366168
21 0.049375534629619314
22 0.0467631674145483
23 0.051506131339594585
24 0.048852830315909014
25 0.05400389349319215
26 0.05030881557563778
27 0.05483883504851244
28 0.050009100123117906
29 0.05344099448129918
30 0.04808426522923427
31 0.050543788607982455
32 0.04540041894680417
33 0.047253524237298845
34 0.0427534022702317
35 0.04429111305614987
36 0.04052030490871995
37 0.04187252037158521
38 0.0387322410185883
39 0.03964179101300076
40 0.03704935080657556
41 0.0374643512409913
42 0.035391813714581874
43 0.03538965045

##### TESTE COM SWISH:

In [11]:
random.seed(8)
minha_mlp_swish = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede, fn_ativacao="swish")

In [12]:
NUM_EPOCAS = 200
TAXA_DE_APRENDIZADO = 0.5

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in x:
        previsao = minha_mlp_swish(exemplo)
        y_pred.append(previsao)

    # loss
    erros = []
    for yt, yp in zip(y_true, y_pred):
        residuo = yp - yt
        erro_quadratico = residuo ** 2
        erros.append(erro_quadratico)        
    loss = (sum(erros))/len(erros)

    # zero grad
    for p in minha_mlp_swish.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in minha_mlp_swish.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

    # mostra resultado (opcional)
    print(epoca, loss.data)

0 0.5959063761944281
1 0.3871558915745604
2 0.20615485075489506
3 0.1464762939964329
4 0.13658688601161662
5 0.13396170082821787
6 0.13224557183173066
7 0.13085451546712715
8 0.1296264622373265
9 0.12848336301412908
10 0.12738270758049294
11 0.12629876622056943
12 0.12521401271283603
13 0.12411495028576738
14 0.12298990641585489
15 0.12182771136906445
16 0.1206167529936587
17 0.11934415170866562
18 0.11799490725958908
19 0.11655090938295634
20 0.11498971059052794
21 0.11328294438901616
22 0.11139424198646215
23 0.10927645881601795
24 0.10686797810217907
25 0.10408784030333924
26 0.10082953624790691
27 0.09695372200423147
28 0.09228145352936434
29 0.08659326293533368
30 0.07964871238505485
31 0.07126060725886904
32 0.06148245073844243
33 0.05093078327226679
34 0.04098852697754623
35 0.03325430427216584
36 0.028341103705910795
37 0.025601367061482062
38 0.024042315420246967
39 0.022984001892922098
40 0.02209228812470658
41 0.02122730210549709
42 0.02033415526300647
43 0.01939152703337675

---

### **DISCUSSÃO DOS RESULTADOS**

A partir das implementações e resultados obtidos durante as épocas com cada função de ativação, foi possível perceber o aprendizado acontecer de formas bem distintas nas diferentes funções. 

No caso da ***ReLU***, foi possível notar um valor constante durante todas as épocas. Isso pode ser um sinal claro de neurônios mortos encontrados na rede - problema grave encontrado comumente nas redes com essa função de ativação. Assim, vemos que para as entradas que disponibilizamos e modelo, essa função não é muito adequada.

Para a ***ELU***, percebe-se uma grande faixa de "altos e baixos" nas primeiras épocas de aprendizado. Entretanto, com o passar das épocas, pode-se notar uma tendência de queda na perda, demonstrando um aprendimento efetivo do modelo com a função - e sem neurônios mortos dessa vez!!!!

Por fim, a ***Swish***, por ser a "mais próxima" na teoria da já conhecida sigmoidal, o aprendizado e queda dos valores de perda observados foi similar, de modo a obter valores interessantes de queda sem muitas variações drásticas como na ELU.

---

### **CONCLUSÃO**

A implementação das funções de ativação ReLU, ELU e Swish demonstrou que é possível alterar uma rede neural a partir somente das suas funções de ativação aplicadas. Os testes mostraram que a rede continua funcional e capaz de aprender durante as épocas, evidenciando a importância da escolha da função de ativação no desempenho e comportamento do modelo. Além disso, as diferenças matemáticas e práticas em relação à função sigmoidal foram observadas, especialmente em relação às derivadas calculadas e a propagação do gradiente.

---

## **REFERÊNCIAS**

**[1]** CASSAR, Daniel. Redes Neurais e Algoritmos Genéticos. 2025. Material de Aula.

**[2]** ESTEVES, Toni. A desvantagem da função ReLU. Medium. 2022. Disponível em: https://estevestoni.medium.com/a-desvantagem-de-utilizar-relu-4478589ef834

**[3]** TORCH. Activation functions source code. GitHub. Disponível em: https://github.com/pytorch/pytorch/blob/96aaa311c0251d24decb9dc5da4957b7c590af6f/torch/nn/modules/activation.py#L422.

**[4]** PAPERS WITH CODE. ELU. Disponível em: https://paperswithcode.com/method/elu. 

**[5]** ALMEIDA, Vitor. Funções de ativação de redes neurais e como são cobradas em concursos. Gran Cursos Online. 2023. Disponível em: https://blog.grancursosonline.com.br/funcoes-de-ativacao-de-redes-neurais-e-como-sao-cobradas-em-concursos/.

**[6]** NIKHADE, Amit. Swish Activation Function. Medium. 2020. Disponível em: https://amitnikhade.medium.com/swish-activation-function-d106fe13930e.